# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split

## Set Hyperparameters

In [2]:
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [3]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [4]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')

## Load Tokenizer and Model

In [5]:
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define Dataset

In [6]:
from pathlib import Path
import yaml 

with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config["device"] = 'cuda' if torch.cuda.is_available() else 'cpu'


data_list = []
file_name_list = []
for file in sorted(list(Path(config['preprocess_data_folder']).glob("*_train.csv")), key=lambda f:f.name):
    if config["ascii_ratio"]+"_" not in file.name:continue
    data = pd.read_csv(file)
    data_list += [data]
    file_name_list += [file.name]
print("num of train data files:", len(data_list))
print("train data files:", file_name_list)
train_data = pd.concat(data_list)

data_list = []
file_name_list = []
for file in sorted(list(Path(config['preprocess_data_folder']).glob("*_valid.csv")), key=lambda f:f.name):
    if config["ascii_ratio"]+"_" not in file.name:continue
    data = pd.read_csv(file)
    data_list += [data]
    file_name_list += [file.name]
print("num of valid data files:", len(data_list))
print("valid data files:", file_name_list)
valid_data = pd.concat(data_list)

dataset_train = train_data
dataset_valid = valid_data

#dataset_train, dataset_valid = train_test_split(data, test_size=0.3, random_state=SEED)

num of train data files: 5
train data files: ['ascii_ratio_higher_20_mlm_btm_train.csv', 'ascii_ratio_higher_20_mlm_train.csv', 'ascii_ratio_higher_20_train.csv', 'ascii_ratio_lower_20_rlm_normal_candidates_train.csv', 'ascii_ratio_lower_20_rlm_train.csv']
num of valid data files: 3
valid data files: ['ascii_ratio_higher_20_valid.csv', 'ascii_ratio_lower_20_rlm_normal_candidates_valid.csv', 'ascii_ratio_lower_20_rlm_valid.csv']


In [7]:
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

In [8]:
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)

In [9]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Define Metric

In [10]:
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


## Train Model

In [11]:
## for wandb setting
os.environ['WANDB_DISABLED'] = 'true'

In [12]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate= 2e-05,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon=1e-08,
    weight_decay=0.01,
    lr_scheduler_type='linear',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

Step,Training Loss,Validation Loss,F1
100,1.386200,0.921264,0.727617
200,0.746400,0.822524,0.765921


TrainOutput(global_step=268, training_loss=0.9518313194388774, metrics={'train_runtime': 257.141, 'train_samples_per_second': 33.172, 'train_steps_per_second': 1.042, 'total_flos': 2244438057216000.0, 'train_loss': 0.9518313194388774, 'epoch': 2.0})

## Evaluate Model

In [15]:
dataset_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [16]:
model.eval()
preds = []

for idx, sample in tqdm(dataset_test.iterrows(), total=len(dataset_test), desc="Evaluating"):
    inputs = tokenizer(sample['text'], return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

Evaluating:   3%|▎         | 878/30000 [00:13<07:35, 64.00it/s]


KeyboardInterrupt: 

In [ ]:
dataset_test['target'] = preds
dataset_test.to_csv(os.path.join(BASE_DIR, 'output.csv'), index=False)